In [1]:
!pip install ipython-sql

In [2]:
import pandas as pd
import sqlite3

In [3]:
users = pd.read_csv('cleaned_users.csv')
transactions = pd.read_csv('cleaned_transactions.csv')
products = pd.read_csv('cleaned_products.csv')

In [4]:
# Create a connection to an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Load DataFrames into SQLite tables
users.to_sql('Users', conn, index=False, if_exists='replace')
print(users.head())

                         ID         CREATED_DATE           BIRTH_DATE STATE  \
0  5ef3b4f17053ab141787697d  2020-06-24 20:17:54  2000-08-11 00:00:00    CA   
1  5ff220d383fcfc12622b96bc  2021-01-03 19:53:55  2001-09-24 04:00:00    PA   
2  6477950aa55bb77a0e27ee10  2023-05-31 18:42:18  1994-10-28 00:00:00    FL   
3  658a306e99b40f103b63ccf8  2023-12-26 01:46:22  1900-01-01 00:00:00    NC   
4  653cf5d6a225ea102b7ecdc2  2023-10-28 11:51:50  1972-03-19 00:00:00    PA   

  LANGUAGE  GENDER  
0   es-419  female  
1       en  female  
2   es-419  female  
3       en      na  
4       en  female  


In [5]:
transactions.to_sql('transactions', conn, index=False, if_exists='replace')
print(transactions.head())

                             RECEIPT_ID PURCHASE_DATE  \
0  0000d256-4041-4a3e-adc4-5623fb6e0c99    2024-08-21   
1  0001455d-7a92-4a7b-a1d2-c747af1c8fd3    2024-07-20   
2  00017e0a-7851-42fb-bfab-0baa96e23586    2024-08-18   
3  000239aa-3478-453d-801e-66a82e39c8af    2024-06-18   
4  00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1    2024-07-04   

                 SCAN_DATE STORE_NAME                   USER_ID       BARCODE  \
0  2024-08-21 14:19:06.539    WALMART  63b73a7f3d310dceeabd4758   15300014978   
1  2024-07-20 09:50:24.206       ALDI  62c08877baa38d1a1f6c211a             0   
2  2024-08-19 15:38:56.813    WALMART  60842f207ac8b7729e472020   78742229751   
3  2024-06-19 11:03:37.468  FOOD LION  63fcd7cea4f8442c3386b589  783399746536   
4  2024-07-05 15:56:43.549   RANDALLS  6193231ae9b3d75037b0f928   47900501183   

   FINAL_QUANTITY  FINAL_SALE  
0             1.0        0.00  
1             0.0        1.49  
2             1.0        0.00  
3             0.0        3.49  
4         

In [6]:
products.to_sql('products', conn, index=False, if_exists='replace')
print(products.head())

          CATEGORY_1              CATEGORY_2                   CATEGORY_3  \
0  Health & Wellness           Sexual Health  Conductivity Gels & Lotions   
1             Snacks           Puffed Snacks         Cheese Curls & Puffs   
2  Health & Wellness               Hair Care        Hair Care Accessories   
3  Health & Wellness               Oral Care                   Toothpaste   
4  Health & Wellness  Medicines & Treatments               Essential Oils   

  CATEGORY_4                                       MANUFACTURER  \
0        NaN                                                NaN   
1        NaN                                                NaN   
2        NaN                           PLACEHOLDER MANUFACTURER   
3        NaN                                  COLGATE-PALMOLIVE   
4        NaN  MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...   

             BRAND       BARCODE  
0              NaN  796494407820  
1              NaN   23278011028  
2          ELECSOP  461817824

In [7]:
#What are the top 5 brands by receipts scanned among users 21 and over?
query = """
SELECT p.brand, COUNT(t.receipt_id) AS receipts_scanned
FROM transactions t
JOIN users u ON t.user_id = u.id
JOIN products p ON t.barcode = p.barcode
WHERE (julianday('now') - julianday(u.birth_date)) / 365.25 >= 21
AND p.brand IS NOT NULL
GROUP BY p.brand
ORDER BY receipts_scanned DESC
LIMIT 6;
"""

In [8]:
# Execute the query
result = pd.read_sql_query(query, conn)

# Display the result
print(result)

                       BRAND  receipts_scanned
0  ANNIE'S HOMEGROWN GROCERY               552
1                       DOVE               535
2                   BAREFOOT               529
3                      ORIBE               483
4              SHEA MOISTURE               460
5                     AVEENO               460


In the above query results, both Aveeno and Shea Moisture tied for the 5th position, each with 460 receipts scanned among users aged 21 and over. This suggests that both brands have a similar level of engagement within this demographic, highlighting their comparable popularity in terms of receipt scans.

In [9]:
# What are the top 5 brands by sales among users that have had their account for at least six months?
query1 = """
WITH eligible_users AS (
    SELECT id AS user_id
    FROM users
    WHERE julianday('now') - julianday(created_date) >= 182.5
),
user_transactions AS (
    SELECT 
        t.barcode,
        t.final_sale,
        p.brand
    FROM transactions t
    JOIN products p ON t.barcode = p.barcode
    JOIN eligible_users eu ON t.user_id = eu.user_id
    WHERE p.brand IS NOT NULL
)
SELECT 
    brand, 
    SUM(final_sale) AS total_sales
FROM user_transactions
GROUP BY brand
ORDER BY total_sales DESC
LIMIT 5;
"""

In [10]:
# Execute the query
result1 = pd.read_sql_query(query1, conn)

# Display the result
print(result1)

                       brand  total_sales
0  ANNIE'S HOMEGROWN GROCERY      2321.04
1                       DOVE      2267.21
2                   BAREFOOT      2224.33
3                      ORIBE      2030.91
4              SHEA MOISTURE      1934.20


In [11]:
#What is the percentage of sales in the Health & Wellness category by generation?
query2 = """
WITH user_generations AS (
    SELECT 
        id AS user_id,
        CASE
            WHEN birth_date <= DATE('1945-12-31') THEN 'Silent Generation'
            WHEN birth_date BETWEEN DATE('1946-01-01') AND DATE('1964-12-31') THEN 'Baby Boomers'
            WHEN birth_date BETWEEN DATE('1965-01-01') AND DATE('1980-12-31') THEN 'Generation X'
            WHEN birth_date BETWEEN DATE('1981-01-01') AND DATE('1996-12-31') THEN 'Millennials'
            WHEN birth_date >= DATE('1997-01-01') THEN 'Generation Z'
            ELSE 'Unknown'
        END AS generation
    FROM users
),
health_and_wellness_sales AS (
    SELECT 
        ug.generation,
        SUM(t.final_quantity * t.final_sale) AS total_health_wellness_sales
    FROM transactions t
    JOIN products p ON t.barcode = p.barcode
    JOIN user_generations ug ON t.user_id = ug.user_id
    WHERE p.category_1 = 'Health & Wellness'
    GROUP BY ug.generation
),
total_sales AS (
    SELECT 
        ug.generation,
        SUM(t.final_quantity * t.final_sale) AS total_sales
    FROM transactions t
    JOIN products p ON t.barcode = p.barcode
    JOIN user_generations ug ON t.user_id = ug.user_id
    GROUP BY ug.generation
)
SELECT 
    ts.generation,
    COALESCE(hws.total_health_wellness_sales, 0) AS total_health_wellness_sales,
    ts.total_sales,
    ROUND((COALESCE(hws.total_health_wellness_sales, 0) * 100.0) / ts.total_sales, 2) AS health_wellness_percentage
FROM total_sales ts
LEFT JOIN health_and_wellness_sales hws ON ts.generation = hws.generation
ORDER BY health_wellness_percentage DESC;
"""


In [12]:
# Execute the query
result2 = pd.read_sql_query(query2, conn)

# Display the result
print(result2)

          generation  total_health_wellness_sales  total_sales  \
0       Baby Boomers                     35268.94     60918.15   
1        Millennials                     79155.31    136711.44   
2       Generation X                     27582.43     47657.91   
3       Generation Z                         0.00         1.97   
4  Silent Generation                         0.00         6.71   

   health_wellness_percentage  
0                       57.90  
1                       57.90  
2                       57.88  
3                        0.00  
4                        0.00  


In [13]:
#Who are Fetch’s power users?
query3 = """
SELECT 
    user_id,
    state,
    language,
    gender,
    transaction_count,
    total_sales
FROM (
    SELECT 
        u.id AS user_id,
        u.state,
        u.language,
        u.gender,
        COUNT(t.receipt_id) AS transaction_count,
        SUM(t.final_sale) AS total_sales
    FROM 
        Users u
    JOIN 
        Transactions t
    ON 
        u.id = t.user_id
    GROUP BY 
        u.id, u.state, u.language, u.gender
    HAVING 
        COUNT(t.receipt_id) >= 2 AND SUM(t.final_sale) > 30
) subquery
ORDER BY 
    total_sales DESC, transaction_count DESC;
"""

The above query defines power users based on two main criteria: a minimum transaction count of at least two purchases and total sales exceeding $30. This assumes that power users exhibit consistent buying behavior, evidenced by multiple transactions, and demonstrate higher engagement through significant spending. Additionally, the query includes user demographic information such as state, language, and gender, which are not directly tied to the definition of power users but can provide valuable insights for further segmentation or profiling. These attributes could be leveraged for targeted marketing strategies or deeper business analysis, allowing the company to understand and engage high-value users more effectively.

In [14]:
# Execute the query
result3 = pd.read_sql_query(query3, conn)

# Display the result
print(result3)

                    user_id state language  gender  transaction_count  \
0  643059f0838dd2651fb27f50    PA       en    male                  4   
1  62ffec490d9dbaff18c0a999    NY       en  female                  6   
2  5f4c9055e81e6f162e3f6fa8    FL   es-419  female                  2   
3  5d191765c8b1ba28e74e8463    TX       en  female                  2   

   total_sales  
0        75.99  
1        52.28  
2        37.96  
3        34.96  


In [15]:
# Execute the query to get the max transaction count
query_max_transaction_count = """
SELECT MAX(transaction_count) AS max_transaction_count
FROM (
    SELECT 
        u.id AS user_id,
        COUNT(t.receipt_id) AS transaction_count
    FROM 
        Users u
    JOIN 
        Transactions t
    ON 
        u.id = t.user_id
    GROUP BY 
        u.id
    HAVING 
        COUNT(t.receipt_id) >= 2 AND SUM(t.final_sale) > 30
) subquery;
"""

# Execute the query
max_transaction_count = pd.read_sql_query(query_max_transaction_count, conn)

# Display the result
print(max_transaction_count)

   max_transaction_count
0                      6


In [16]:
#Which is the leading brand in the Dips & Salsa category?
query4 = """
SELECT 
    p.brand AS leading_brand,
    SUM(t.final_sale) AS total_sales
FROM 
    Transactions t
JOIN 
    Products p
ON 
    t.barcode = p.barcode
WHERE 
    (LOWER(p.category_1) = 'dips & salsa' 
    OR LOWER(p.category_2) = 'dips & salsa' 
    OR LOWER(p.category_3) = 'dips & salsa' 
    OR LOWER(p.category_4) = 'dips & salsa')
    AND p.brand IS NOT NULL
    
GROUP BY 
    p.brand
ORDER BY 
    total_sales DESC
LIMIT 1;
"""

In [17]:
# Execute the query
result4 = pd.read_sql_query(query4, conn)

# Display the result
print(result4)

  leading_brand  total_sales
0      TOSTITOS    103104.29


In [18]:
#At what percent has Fetch grown year over year?
query5 = """
WITH YearlySales AS (
    SELECT 
        strftime('%Y', t.purchase_date) AS year,
        SUM(t.final_sale) AS total_sales
    FROM 
        Transactions t
    GROUP BY 
        strftime('%Y', t.purchase_date)
    ORDER BY 
        year
),
Growth AS (
    SELECT 
        y1.year AS current_year,
        y1.total_sales AS current_year_sales,
        y2.total_sales AS previous_year_sales,
        CASE 
            WHEN y2.total_sales IS NOT NULL THEN 
                ((y1.total_sales - y2.total_sales) * 100.0 / y2.total_sales)
            ELSE 
                NULL
        END AS growth_percentage
    FROM 
        YearlySales y1
    LEFT JOIN 
        YearlySales y2
    ON 
        CAST(y1.year AS INTEGER) = CAST(y2.year AS INTEGER) + 1
)
SELECT 
    current_year,
    COALESCE(previous_year_sales, 0) AS previous_year_sales,
    current_year_sales,
    growth_percentage
FROM 
    Growth
ORDER BY 
    current_year;
"""

In [19]:
# Execute the query
result5 = pd.read_sql_query(query5, conn)

# Display the result
print(result5)

  current_year  previous_year_sales  current_year_sales growth_percentage
0         2024                    0           171182.79              None
